# Clasificación de Dígitos MNIST - Versión Compatible

Versión actualizada para generar modelo compatible con versiones modernas de TensorFlow

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

import logging
from tensorflow.keras.preprocessing.image import ImageDataGenerator

logger = tf.get_logger()
logger.setLevel(logging.ERROR)


In [ ]:

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [ ]:
train_images = []
train_labels = []

for img, label in ds_train:
    train_images.append(img.numpy())
    train_labels.append(label.numpy())

train_images = np.array(train_images)
train_labels = np.array(train_labels)

train_images = train_images / 255.0
train_images = np.expand_dims(train_images, -1)
train_labels=np.array(train_labels)


test_images = []
test_labels = []

for img, label in ds_test:
    test_images.append(img.numpy())
    test_labels.append(label.numpy())

test_images = np.array(test_images) / 255.0
test_images = np.expand_dims(test_images, -1)
test_labels = np.array(test_labels)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=[0.8,1.2]
)

train_gen = datagen.flow(train_images, train_labels, batch_size=32)


In [ ]:


model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),padding='same',activation='relu'),
   
    tf.keras.layers.MaxPooling2D((2,2)),
    
    tf.keras.layers.Conv2D(64, (3,3),padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
   
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:

history = model.fit(
    train_gen,
    epochs=10,
    validation_data=(test_images, test_labels),
    verbose=1
)

In [ ]:


test_loss, test_accuracy = model.evaluate(
    test_images,test_labels,verbose=1
)

model.save('modelo_MINST_completo.keras')

loaded_model = tf.keras.models.load_model('modelo_MINST_completo.keras')

loaded_test_loss, loaded_test_accuracy = loaded_model.evaluate(test_images, test_labels, verbose=1)